# Week 1:
## Monday May 20, 2019:
* first day!
* setup and luncheon
* researched Roary

**Brief project history:**

Thomas Pedersen was a UAMS PhD student who originally created [FindMyFriends](https://github.com/thomasp85/FindMyFriends) to be able to rapidly construct the pangenome (core and accessory genes) across multliple bacterial genomes quickly and accurately. The program first uses an algorithm called [CD-Hit](https://www.ncbi.nlm.nih.gov/pubmed/11294794) to quickly group the genes into broad groups, and then the gene clusters are split into smaller clusters based on k-mer similarity and neighboring sequences (to separate orthologues (difference from speciation) from paralogues (difference from duplication)). Once the genes are grouped, it can be determined to what degree certain genes are shared between the genomes that were fed to the algorithm. 

While Thomas was working on FindMyFriends, a separate pan-genome construction algorithm was published called [Roary](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4817141/). Roary accomplished the same task faster, so Thomas decided to use design elements like CD-Hit from Roary to improve the speed of FindMyFriends. Once Thomas graduated with his PhD, FindMyFriends was left in an unfinished state. *Consequently, the goal of this project is to replace the CD-Hit pre-clustering step with a kmer-counting algorithm to try to restore functionality and accuracy to FindMyFriends.*

**Research Notes:**

Programs like PanOCT and PGAP run all-against-all BLASTs across the input genomes, but this scales quadratically with computation time and memory usage, so it becomes impractical to use these programs for large amounts of data. Roary (and FindMyFriends) instead use kmer counting. If two sequences are similar, they must exactly share a certain number of short sequences (kmers). In the original CD-Hit paper, they provide an example: "[T]wo sequences having 85% identical residues over a 100-residue window will have at least 70 identical dipeptides,55 tripeptides and 25 pentapeptides" (Li et al., 2001). Sequences can then be quickly compared and grouped so that a BLAST does not have to be run for all of them. The problem with the CD-Hit algorithm (as used in the latest version of FindMyFriends) is that once it groups the genomes based on kmer counts, it randomly chooses a representative sample from each group to be BLASTed against other groups. This eliminates reproducible results when running the algorithm. 

**References**

Page AJ, Cummins CA, Hunt M, et al. Roary: rapid large-scale prokaryote pan genome analysis. Bioinformatics. 2015;31(22):3691–3693. doi:10.1093/bioinformatics/btv421

Weizhong Li, Lukasz Jaroszewski, Adam Godzik. Clustering of highly homologous sequences to reduce the size of large protein databases. Bioinformatics. 2001;17(3):282–283, https://doi.org/10.1093/bioinformatics/17.3.282
***

## Tuesday May 21, 2019:
* got files from Kaleb
* downloaded prodigal, RStudio
* generated heatmap of clusters of genes from a single genome
* downloaded 22 genomes (incorrectly)

**Generating .faa files**

Create a text file containing assemblies that are to be downloaded with one assembly identifier on each line. Go to [Batch Entrez](https://www.ncbi.nlm.nih.gov/sites/batchentrez) and upload the file to retrieve the .fna files from GenBank (not RefSeq!) for each assembly. Run prodigal to convert the .fna files to .faa files:

**prodigal -i inputfilename.fna -a outputfilename.faa**

For the single genome heatmap, I used the GCA_00369165.2 chromosome assembly from NCBI. 
For the 25 genomes, I used:
* GCA_000092525.1
* GCA_000006925.2
* GCA_000006665.1
* GCA_000008865.2
* GCA_000019645.1
* GCA_000176695.2
* GCA_000012005.1
* GCA_000176575.2
* GCA_000350785.1
* GCA_000176655.2
* GCA_000026325.2
* GCA_000091005.1
* GCA_000010485.1
* GCA_000007445.1
* GCA_000690815.1
* GCA_000734955.1
* GCA_000613265.1
* GCA_000333215.1
* GCA_000714595.1
* GCA_000167875.1
* GCA_000012025.1
* GCA_000005845.2
* GCA_000010245.1
* GCA_002007705.1
* GCA_900092615.1

**Generating the heatmap**

In [ ]:
library("ape")
library("kmer")
library("MASS")
setwd("C:/Users/Matthew/Documents/UAMS SURF/K-mer testing/FAA files")

aminoAcids <- read.FASTA("GCF_000005845.2_ASM584v2_genomic.faa", type = "AA")
# create a distance matrix using 4-mers from each protein from the single genome
aminoAcidsDist <- kdistance(aminoAcids, k = 4)
# roughly group the proteins into a dendrogram
aminoAcidsTree <- as.dendrogram(hclust(aminoAcidsDist), "average")
plot(heatmap(as.matrix(aminoAcidsDist), Rowv=aminoAcidsTree, Colv = "Rowv", scale = "none"))
# optionally print out the kmer distance matrix
write.matrix(aminoAcidsDist, file = "file_name", sep= " ")

My run using the GCA_00369165.2 chromosome with k=4 generated the heatmap below. You can see that where the dendrogram goes down many layers, the heatmap area lightens. The sensitivity of the heatmap can be changed to show larger, clearer clusters, but this was just a test to see if a heatmap could be generated.
![heatmap](images/AA4merHeatMap_GCA_003697165.2.png)

## Wednesday May 22, 2019:
* setup jupyter notebook
* download 25 genomes from GenBank (correctly this time)
* work on getting top 5 9-mers for each protein in a single genome

Today consisted of a lot of setup and troubleshooting with RStudio while trying to working on the 9-mers. I was trying to create a large matrix of kmer counts for the proteins of a genome using the kmer R package, but my computer could not do it for all of the proteins at once. Code:


In [ ]:
library("ape")
library("kmer")
library("MASS")
setwd("C:/Users/Matthew/Documents/UAMS SURF/K-mer testing/FAA files")
proteins <- read.FASTA("GCA_000005845.2_ASM584v2.faa", type = "AA")
kmerCounts <- kcount(proteins[0:200], k = 9)

I was able to get the kcount to work for 200 of the proteins, and it produced a 200 x 10077696 matrix. The rows were the 200 protein sequences and there were n^k columns, where n is the alphabet size (n = 6 for reduced amino acid classes: AGPST, C, DENQ, FWY, HKR, ILMV) and k is the kmer length (k = 9 for this example). If I access the matrix using **kmerCounts[1,1:20]**, I am able to see the kmer counts for specific kmers for the first sequence. 

![kmerCounts](images/kmerCount.jpg)


Tomorrow, the goal is to sort the columns for each row to get the 5 top occuring kmers for each sequence.